# <p align="center"><b style="color:#1f6feb;">🧠📊Comparative Study of LSTM and Transformer Models for Neural Question Answering📊🧠</b></p>


### ‎‧₊**˚✿[My Name]✿˚** : **[Mohamed Reda Ramadan Khamis]**
### ‎‧₊**˚✿[My Phone Number]✿˚** : **[01554725661]**

# **LSTM QA**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pokhrelarahanta/subjective-question-answer-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.90M/1.90M [00:00<00:00, 3.22MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/pokhrelarahanta/subjective-question-answer-dataset/versions/1


In [ ]:
df = pd.read_csv(path + '/QA_dataset.csv')
df.head()

,Paragraphs,Question1,Question2,Question3,Answer1,Answer2,Answer3
0,Microorganisms in the environment are diverse ...,Why are viruses different from other microbes?,How do viruses cause infections in living orga...,What makes protozoa significant in environment...,Viruses consist solely of nucleic acids and pr...,Viruses infect and replicate inside living cel...,Protozoa serve as significant sources of patho...
1,Microorganisms in the environment encompass di...,How are viruses visualized due to their small ...,What is the mode of transmission for norovirus...,How are bacteria classified based on cellular ...,"Viruses, which are submicroscopic, require ele...",Noroviruses can be transmitted via ingestion o...,Bacteria are classified as prokaryotes since t...
2,"Bacteria, though simple, play crucial roles in...",What roles do bacteria play in the environment?,How do viruses replicate if they lack ribosome...,"Why are fungi, algae, and protozoa considered ...",Bacteria play crucial roles in the environment...,Viruses must infect and replicate inside livin...,"Fungi, algae, and protozoa are eukaryotic sinc..."
3,"Actinomycetes, externally resembling miniature...",What are the key characteristics of actinomyce...,How do bacteria differ from more complex micro...,Why are bacteria essential for life on Earth?,"Actinomycetes resemble miniature fungi, produc...",Bacteria lack developed internal structures an...,Bacteria play crucial roles in various environ...
4,"Protozoa, unicellular eukaryotes with characte...",What sets protozoa apart from bacteria?,How do protozoa adapt to various environments?,Why are some protozoa considered threats to dr...,Protozoa are unicellular eukaryotes with chara...,Protozoa like Giardia and Cryptosporidium can ...,Protozoa like Giardia and Cryptosporidium can ...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4118 entries, 0 to 4117
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Paragraphs  4118 non-null   object
 1   Question1   4118 non-null   object
 2   Question2   4118 non-null   object
 3   Question3   4118 non-null   object
 4   Answer1     4118 non-null   object
 5   Answer2     4118 non-null   object
 6   Answer3     4118 non-null   object
dtypes: object(7)
memory usage: 225.3+ KB


In [ ]:
# Transform the dataset: Combine context + question and associate it with the answer
flattened_data = {
    "input_text": [],
    "target_text": []
}

for _, row in df.iterrows():
    for i in range(1, 3 + 1):  # for Question1, Question2, Question3
        question = row[f"Question{i}"]
        answer = row[f"Answer{i}"]
        context = row["Paragraphs"]

        input_text = f"{context} [SEP] {question}"
        target_text = f"<start> {answer} <end>"

        flattened_data["input_text"].append(input_text)
        flattened_data["target_text"].append(target_text)

flat_df = pd.DataFrame(flattened_data)


In [ ]:
flat_df

,input_text,target_text
0,Microorganisms in the environment are diverse ...,<start> Viruses consist solely of nucleic acid...
1,Microorganisms in the environment are diverse ...,<start> Viruses infect and replicate inside li...
2,Microorganisms in the environment are diverse ...,<start> Protozoa serve as significant sources ...
3,Microorganisms in the environment encompass di...,"<start> Viruses, which are submicroscopic, req..."
4,Microorganisms in the environment encompass di...,<start> Noroviruses can be transmitted via ing...
...,...,...
12349,Viruses lack ribosomes and the ability to meta...,<start> Noroviruses spread through consuming c...
12350,Viruses lack ribosomes and the ability to meta...,<start> Noroviruses attach to and infect intes...
12351,Bacteria are single-celled organisms categoriz...,<start> Actinomycetes are specialized prokaryo...
12352,Bacteria are single-celled organisms categoriz...,<start> Actinomycetes are essential for antibi...


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
flat_df = flat_df
# 1. Tokenizer setup
input_tokenizer = Tokenizer(filters='', lower=True, oov_token="<OOV>")
target_tokenizer = Tokenizer(filters='', lower=True, oov_token="<OOV>")

# 2. Fit on texts
input_tokenizer.fit_on_texts(flat_df["input_text"])
target_tokenizer.fit_on_texts(flat_df["target_text"])

# 3. Convert to sequences
input_sequences = input_tokenizer.texts_to_sequences(flat_df["input_text"])
target_sequences = target_tokenizer.texts_to_sequences(flat_df["target_text"])

# 4. Get max lengths
max_encoder_seq_length = max(len(seq) for seq in input_sequences)
max_decoder_seq_length = max(len(seq) for seq in target_sequences)

# 5. Pad sequences
encoder_input_data = pad_sequences(input_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences([seq[:-1] for seq in target_sequences], maxlen=max_decoder_seq_length - 1, padding='post')
decoder_target_data = pad_sequences([seq[1:] for seq in target_sequences], maxlen=max_decoder_seq_length - 1, padding='post')


In [ ]:
max_decoder_seq_length

98

In [ ]:
max_encoder_seq_length

462

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Vocabulary sizes
num_encoder_tokens = len(input_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1

# Hyperparameters
embedding_dim = 128
lstm_units = 500

# ========== Encoder ==========
encoder_inputs = Input(shape=(None,), name="encoder_inputs")
encoder_embedding_layer = Embedding(input_dim=num_encoder_tokens, output_dim=embedding_dim, name="encoder_embedding")
encoder_embedding = encoder_embedding_layer(encoder_inputs)
_, state_h, state_c = LSTM(lstm_units, return_state=True, name="encoder_lstm")(encoder_embedding)
encoder_states = [state_h, state_c]

# ========== Decoder ==========
decoder_inputs = Input(shape=(None,), name="decoder_inputs")
decoder_embedding_layer = Embedding(input_dim=num_decoder_tokens, output_dim=embedding_dim, name="decoder_embedding")
decoder_embedding = decoder_embedding_layer(decoder_inputs)

decoder_lstm_layer = LSTM(lstm_units, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm_layer(decoder_embedding, initial_state=encoder_states)

decoder_dense_layer = Dense(num_decoder_tokens, activation='softmax', name="decoder_dense")
decoder_outputs = decoder_dense_layer(decoder_outputs)

# ========== Define Full Model ==========
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Optional: Print summary
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, None, 128) │  5,674,624 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, None, 128) │  3,311,232 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 500),     │  1,258,000 │ encoder_embeddin… │
│                     │ (None, 500),      │            │                   │
│                     │ (None, 500)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │  1,258,000 │ decoder_embeddin… │
│                     │ 500), (None,      │            │ encoder_lstm[0][… │
│                     │ 500), (None,      │            │ encoder_lstm[0][… │
│                     │ 500)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None,      │ 12,960,369 │ decoder_lstm[0][… │
│ (Dense)             │ 25869)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,462,225 (93.32 MB)

 Trainable params: 24,462,225 (93.32 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Ensure target output is 3D for sparse_categorical_crossentropy
decoder_target_data = decoder_target_data.reshape(
    decoder_target_data.shape[0],
    decoder_target_data.shape[1],
    1
)

# Train the model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=5,
    validation_split=0.2
)


Epoch 1/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 66s 208ms/step - loss: 2.9460 - val_loss: 2.0427
Epoch 2/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 66s 213ms/step - loss: 1.7433 - val_loss: 1.9909
Epoch 3/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 67s 217ms/step - loss: 1.6698 - val_loss: 1.9592
Epoch 4/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 67s 217ms/step - loss: 1.5963 - val_loss: 1.9260
Epoch 5/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 67s 217ms/step - loss: 1.5171 - val_loss: 1.8984


In [ ]:
# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference inputs
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Re-declare decoder input for inference
decoder_inputs_inf = Input(shape=(None,))

#  Use the original decoder embedding layer (not its output)
decoder_emb_inf = decoder_embedding_layer(decoder_inputs_inf)

#  Reuse the original LSTM and Dense layers
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm_layer(
    decoder_emb_inf, initial_state=decoder_states_inputs)
decoder_states_inf = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense_layer(decoder_outputs_inf)

# Final decoder inference model
decoder_model = Model(
    [decoder_inputs_inf] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf
)


In [ ]:
def decode_answer(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.array([[target_tokenizer.word_index['<start>']]])
    decoded_sentence = ""
    stop_condition = False
    max_len = max_decoder_seq_length

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = next((w for w, i in target_tokenizer.word_index.items() if i == sampled_token_index), "")

        print("Sampled:", sampled_word)

        if sampled_word in ["<end>", "", None] or len(decoded_sentence.split()) > max_len:
            stop_condition = True
        else:
            decoded_sentence += " " + sampled_word
            target_seq = np.array([[sampled_token_index]])
            states_value = [h, c]

    return decoded_sentence.strip()


In [ ]:
def answer_question(context, question):
    input_text = f"{context} [SEP] {question}"
    seq = input_tokenizer.texts_to_sequences([input_text])
    padded = pad_sequences(seq, maxlen=max_encoder_seq_length, padding='post')
    return decode_answer(padded)

# Example:
print(answer_question(
    "Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds. In humans and birds, they cause respiratory tract infections that can range from mild to lethal. Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS and COVID-19. In cows and pigs, they cause diarrhea; while in mice, they cause hepatitis and encephalomyelitis.",
    "What is Coronaviruses "

))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Sampled: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Sampled: main
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sampled: types
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Sampled: of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Sampled: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Sampled: atmosphere
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sampled: is
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sampled: a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sampled: significant
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Sampled: role
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sampled: in
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Sampled: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Sampled: atmosphere
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sampled: and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Sampled: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Sampled: atmosphere
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Sampled: and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sampled: the
1/1 ━━━━━━━━━━

# **Transformers QA**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pokhrelarahanta/subjective-question-answer-dataset")

print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv(path + '/QA_dataset.csv')
df.head()

In [ ]:
df.info()

In [ ]:
# Transform the dataset: Combine context + question and associate it with the answer
flattened_data = {
    "input_text": [],
    "target_text": []
}

for _, row in df.iterrows():
    for i in range(1, 3 + 1):  # for Question1, Question2, Question3
        question = row[f"Question{i}"]
        answer = row[f"Answer{i}"]
        context = row["Paragraphs"]

        input_text = f"{context} [SEP] {question}"
        target_text = f"<start> {answer} <end>"

        flattened_data["input_text"].append(input_text)
        flattened_data["target_text"].append(target_text)

flat_df = pd.DataFrame(flattened_data)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Sample subset
flat_df = flat_df

# Tokenizers
input_tokenizer = Tokenizer(filters='', lower=True, oov_token='<OOV>')
target_tokenizer = Tokenizer(filters='', lower=True, oov_token='<OOV>')

input_tokenizer.fit_on_texts(flat_df["input_text"])
target_tokenizer.fit_on_texts(flat_df["target_text"])

input_sequences = input_tokenizer.texts_to_sequences(flat_df["input_text"])
target_sequences = target_tokenizer.texts_to_sequences(flat_df["target_text"])

max_len_src = max(len(seq) for seq in input_sequences)
max_len_tgt = max(len(seq) for seq in target_sequences)

encoder_input = pad_sequences(input_sequences, maxlen=max_len_src, padding='post')
decoder_input = pad_sequences([seq[:-1] for seq in target_sequences], maxlen=max_len_tgt - 1, padding='post')
decoder_target = pad_sequences([seq[1:] for seq in target_sequences], maxlen=max_len_tgt - 1, padding='post')

vocab_size_src = len(input_tokenizer.word_index) + 1
vocab_size_tgt = len(target_tokenizer.word_index) + 1

decoder_target = decoder_target.reshape(decoder_target.shape[0], decoder_target.shape[1], 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add
from tensorflow.keras.models import Model

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model):
        super().__init__()
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(d_model)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.constant(angle_rads[np.newaxis, ...], dtype=tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

def transformer_block(x, context, head_size, num_heads, ff_dim, dropout=0.1):
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, context)
    attn = Dropout(dropout)(attn)
    out1 = LayerNormalization(epsilon=1e-6)(Add()([x, attn]))
    ffn = Dense(ff_dim, activation='relu')(out1)
    ffn = Dense(x.shape[-1])(ffn)
    ffn = Dropout(dropout)(ffn)
    return LayerNormalization(epsilon=1e-6)(Add()([out1, ffn]))

def build_transformer(vocab_size_src, vocab_size_tgt, max_len_src, max_len_tgt, d_model=128, ff_dim=512, num_heads=4):
    enc_inputs = Input(shape=(None,))
    dec_inputs = Input(shape=(None,))

    src_emb = Embedding(vocab_size_src, d_model)(enc_inputs)
    src_emb = PositionalEncoding(max_len_src, d_model)(src_emb)

    tgt_emb = Embedding(vocab_size_tgt, d_model)(dec_inputs)
    tgt_emb = PositionalEncoding(max_len_tgt, d_model)(tgt_emb)

    enc_out = transformer_block(src_emb, src_emb, d_model, num_heads, ff_dim)
    dec_out = transformer_block(tgt_emb, enc_out, d_model, num_heads, ff_dim)

    output = Dense(vocab_size_tgt, activation='softmax')(dec_out)
    return Model(inputs=[enc_inputs, dec_inputs], outputs=output)


In [ ]:
# Build model
model = build_transformer(
    vocab_size_src=vocab_size_src,
    vocab_size_tgt=vocab_size_tgt,
    max_len_src=max_len_src,
    max_len_tgt=max_len_tgt,
    d_model=128,
    ff_dim=512,
    num_heads=4
)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train
model.fit(
    [encoder_input, decoder_input],
    decoder_target,
    batch_size=32,
    epochs=10,
    validation_split=0.1
)


In [ ]:
model.summary()

In [ ]:
def decode_answer(context, question):
    input_text = f"{context.strip()} [SEP] {question.strip()}"
    input_seq = input_tokenizer.texts_to_sequences([input_text])
    enc_input = pad_sequences(input_seq, maxlen=max_len_src, padding='post')

    start_token = target_tokenizer.word_index['<start>']
    end_token = target_tokenizer.word_index['<end>']
    output_seq = [start_token]

    for _ in range(max_len_tgt):
        dec_input = pad_sequences([output_seq], maxlen=max_len_tgt - 1, padding='post')
        pred = model.predict([enc_input, dec_input], verbose=0)
        next_token = np.argmax(pred[0, len(output_seq) - 1])
        if next_token == end_token or next_token == 0:
            break
        output_seq.append(next_token)

    reverse_tgt_index = {i: w for w, i in target_tokenizer.word_index.items()}
    decoded_words = [reverse_tgt_index.get(i, '') for i in output_seq[1:]]
    return ' '.join(decoded_words).strip()


In [ ]:
context = "Microorganisms in the environment are diverse in origin and adapted to harsh, fluctuating conditions."
question = "Why are viruses different from other microbes?"
print(decode_answer(context, question))
